In [ ]:
#Get the models
from transformers import AutoModelForSequenceClassification
from transformers import TFAutoModelForSequenceClassification
from transformers import AutoTokenizer, AutoConfig
import numpy as np
import pandas as pd
from scipy.special import softmax
from sklearn.metrics import accuracy_score
MODEL = f"cardiffnlp/twitter-roberta-base-sentiment-latest" #Reference to model
tokenizer = AutoTokenizer.from_pretrained(MODEL) #Get preset tokenizer
config = AutoConfig.from_pretrained(MODEL) #Get preset configs
model = AutoModelForSequenceClassification.from_pretrained(MODEL) #Get the model

def evaluate_text(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    label = ''
    score = 0
    for i in range(scores.shape[0]):
        s = scores[ranking[i]]
        if s > score:
            score = s
            label = config.id2label[ranking[i]]
    return label

def evaluate_text_cScore(text):
    encoded_input = tokenizer(text, return_tensors='pt')
    output = model(**encoded_input)
    scores = output[0][0].detach().numpy()
    scores = softmax(scores)
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    score = 0
    for i in range(scores.shape[0]):
        s = scores[ranking[i]]
        if s > score:
            score = s
    score = round(score,2)
    return score

#Get the dataset
input_data = pd.read_csv('sentiment_test_cases.csv', encoding='cp1252')
input_data = input_data.dropna()

#Apply calculations
input_data['model_output'] = input_data['text'].apply(evaluate_text)
input_data['confidence_score'] = input_data['text'].apply(evaluate_text_cScore)

#Declare columns to keep
columns_to_keep = ['text', 'expected_sentiment', 'model_output', 'confidence_score']
input_data = input_data[columns_to_keep]

# Save the output to a CSV file
input_data.to_csv('output_sentiment_test.csv', index=False)

#Calculate for accuracy
accuracy = accuracy_score(input_data['expected_sentiment'], input_data['model_output'])
print(f'Accuracy: {accuracy * 100:.2f}%')
